![Erudio logo](../../img/erudio-logo-small.png)

# Decorators

A decorator, at base, is nothing more than syntax sugar for a callable that takes one function or class as an argument, and returns ... **something**.  Given how deeply introspective Python can be, you can modify functions and classes in pretty much any conceivable way (some take more work than others).  In general, decorators are simply a convenient way of expressing a kind of modification that you will potentially want to apply to many functions or classes.

Hopefully without belaboring the matter too much, what a decorator returns really can be *anything*.  Most of the time it is a somewhat modified class or function that performs a largely similar function to the undecorated version.  But the syntax is not so constrained.

In [1]:
def func_to_num(fn):
    return 42

@func_to_num
def fibonacci(max=float('inf'), a=1, b=2):
    while a < max:
        yield a
        a, b = b, a+b
        
print(fibonacci)

42


In [2]:
try:
    fibonacci(999)
except Exception as err:
    print(err)

'int' object is not callable


The decorator `@func_to_num` is a generally terrible decorator with no reasonable purpose.  But it **is** a decorator.

## Transforming Functions

Decorators can be a powerful way of expressing "cross-cutting" behavior that you want to apply to different functions. A very simple, but still useless, decorator is the identity decorator.  It simply returns a function that behaves the same way as the function passed into it.  However, the way we write this shows the structure of writing more useful ones.

In [3]:
def decorate(func):
    def new_func(arg):
        return func(arg)
    return new_func

Even without decorators, We could modify the function using `decorate()` and rebind it to the same name.

In [4]:
def fn1(x):
    return x + 1

print("Original fn1() answer:", fn1(5))

fn1 = decorate(fn1)
print("Modified fn1() answer:", fn1(5))

Original fn1() answer: 6
Modified fn1() answer: 6


It is generally prettier to do a semantically identical thing using a decorator like the following.  Decorators are higher-order functions that modify a function, and then rebind the new function the same name.  Their main advantage is simply that they come at the start of a function definition rather at the end of a long function definition (or elsewhere than that even).

In [5]:
def fn2(x):
    return x + 1

@decorate
def fn3(x):
    return x + 1

The function returned by `@decorate` is bound to the same name `fn3` and replaces the original, decorated function.  In the example above, ```new_func``` becomes the new implementation of ```fn3```. Let's contrast `fn2()` with `fn3()`.

In [6]:
print(f"Undecorated: {fn2.__name__}(17) == {fn2(17)}")
print(f"Decorated: {fn3.__name__}(17) == {fn3(17)}")

Undecorated: fn2(17) == 18
Decorated: new_func(17) == 18


### A first useful decorator

Let's imagine that we want to allow `fun()` to operate on sequences of numbers, but we only want to define it as an operation on a single number.

In [7]:
def map_scalar(func):
    def map_to_seq(*args):
        return list(map(func, args))
    return map_to_seq

In [8]:
@map_scalar
def add_one(x):
    return x + 1

In [9]:
add_one(3)

[4]

In [10]:
add_one(10, 20, 30, 40)

[11, 21, 31, 41]

## Change behavior on duck-type of arguments

A slightly more interesting variant on the above is to turn Python functions into something like NumPy ufuncs.  That is, perhaps we would like them to operate on either scalars or sequences (preserving the sequence type).

In [11]:
def elementwise(fn):
    "Transform a function on scalars into a function on collections"
    
    def newfn(arg):
        "Inner function of elementwise decorator"
        # Treat a string as scalar even though it is iterable
        if isinstance(arg, str):
            return fn(arg)        
        try:                      # Something iterable
            return type(arg)(map(fn, arg))
        except TypeError as err:  # Assume it is scalar
            return fn(arg)
        
    return newfn

In [12]:
@elementwise
def compute(x):
    "Calculate one less than the cube of an input"
    return x**3 - 1

In [13]:
compute(5)

124

In [14]:
compute([1, 2+3j, 3.14]) 

[0, (-47+9j), 29.959144000000002]

In [15]:
compute({8, 9, 10})

{511, 728, 999}

# Where naïve wrapping goes wrong

The simple examples above show a general pattern for creating decorators.  However, they are fragile in the face of object introspection, whether for debugging or other purposes.  Suppose we try to use a function and it goes wrong:

In [16]:
try: 
    compute("Ionesco")
except Exception as err:
    print(repr(err))

TypeError("unsupported operand type(s) for ** or pow(): 'str' and 'int'")


That is not so terrible as an error message, we should try to figure out how we are meant to use the function.

In [17]:
help(compute)

Help on function newfn in module __main__:

newfn(arg)
    Inner function of elementwise decorator



We provided a docstring for `compute()`, but it got lost when it was decorated!  What we actually have is an instance of the inner function created within the decorator; that one does not have any docstring.  The actual function object only has the generic description of what it does within the decorator.

## Using `functools.wraps`

As we do further introspection of `compute` we become more troubled that it is *not* really the function we created.  Fortunately, the solution here is very simple.  We simply need to use `functools.wraps` to cleanup these details for us.

In [18]:
import functools

def elementwise(fn):
    @functools.wraps(fn)  # <-- Add this to the interior function
    def newfn(arg):
        if isinstance(arg, str):  # string as scalar
            return fn(arg)        
        try:                      # Something iterable
            return type(arg)(map(fn, arg))
        except TypeError as err:  # Assume it is scalar
            return fn(arg)
    return newfn

In [19]:
@elementwise
def compute(x):
    "Calculate one less than the cube of input value(s)"
    return x**3 - 1

compute(5), compute((1, 2, 3))

(124, (0, 7, 26))

In [20]:
help(compute)

Help on function compute in module __main__:

compute(x)
    Calculate one less than the cube of input value(s)



In [21]:
compute.__name__

'compute'

## Combining decorators

It is often possible and useful to combine several decorators.  For example, let us make a decorator for logging operations performed.

In [26]:
from datetime import datetime
def log_calls(fn):
    # Name logfile after the function name
    logfile = open(f"{fn.__name__}.log", 'w', encoding="utf-8")
    @functools.wraps(fn)
    def inner(*args, **kws):
        # Inner function accepts arbitrary positional and named args
        result = fn(*args, **kws)
        # Perform the logging
        print(datetime.now().isoformat(), file=logfile, end=" ")
        # To simplify, not logging keyword args in this example
        args = map(str, args)
        print(f"{fn.__name__}({','.join(args)}) → {result}", file=logfile, flush=True)
        return result
    return inner

In [27]:
@log_calls
@elementwise
def compute(x):
    "Calculate one less than the cube of input value(s)"
    return x**3 - 1

In [28]:
from time import sleep
for data in [5, (1, 2, 3), [1, 2+3j, 3.14], {8, 9, 10}]:
    compute(data)
    sleep(1.2)

In [29]:
%%bash
cat compute.log

2024-02-13T15:21:06.482839 compute(5) → 124
2024-02-13T15:21:07.690421 compute((1, 2, 3)) → (0, 7, 26)
2024-02-13T15:21:08.896817 compute([1, (2+3j), 3.14]) → [0, (-47+9j), 29.959144000000002]
2024-02-13T15:21:10.102934 compute({8, 9, 10}) → {728, 999, 511}


# Decorator Factories

A common pattern is to use *decorator factories* rather than decorators directly.  This allows us to parameterize the decoration of a function in some manner. In order to accept these parameters, we have to wrap our decorator in another function that will accept these.  Let's write a decorator factory that will write the output of a function to a file.

In [30]:
from functools import wraps
from datetime import datetime

def log_results(filename):
    def decorator(func):
        @wraps(func)
        def inner(*args, **kws):
            result = func(*args, **kws)
            with open(filename, 'a') as fh:
                now = datetime.utcnow().isoformat()
                name = func.__name__
                fh.write(f"{now}\t{name}\t{args}\t{kws}\t{repr(result)}\n")
            return result
        return inner
    return decorator

logfile = "ine-lesson.log"

In [31]:
@log_results(logfile)
def myfunc(a, b, c=None):
    "Return a formatted display of arguments"
    outstr = f"a={a}; b={b}; c={c}"
    return outstr

In [32]:
# Make sure the docstring of the decorated function remains
help(myfunc)

Help on function myfunc in module __main__:

myfunc(a, b, c=None)
    Return a formatted display of arguments



In [33]:
myfunc(3, 6, "Flowerpot")

'a=3; b=6; c=Flowerpot'

In [34]:
myfunc(4, 5, "Gardenhose")

'a=4; b=5; c=Gardenhose'

We might use the decorator with another function. In this case, it logs to the same file, but a different one is equally possible as a parameter.

In [35]:
@log_results(logfile)
def hypotenuse(a, b):
    "Calculate the hypotenuse of a right triangle"
    from math import sqrt
    c = sqrt(a**2 + b**2)
    return c

In [36]:
hypotenuse(3, 4), hypotenuse(11, 15)

(5.0, 18.601075237738275)

In [37]:
# Lets read the log
with open(logfile) as fi:
    print(''.join(fi.readlines()))

2024-02-13T09:53:25.725204	myfunc	(3, 6, 'Flowerpot')	{}	'a=3; b=6; c=Flowerpot'
2024-02-13T09:53:26.258777	myfunc	(4, 5, 'Gardenhose')	{}	'a=4; b=5; c=Gardenhose'
2024-02-13T09:53:27.546574	hypotenuse	(3, 4)	{}	5.0
2024-02-13T09:53:27.546574	hypotenuse	(11, 15)	{}	18.601075237738275



When we use a function based approach, we need to nest several levels.  There is the factory function, inside that is the (parameterized)

# Decorator Classes

You can use a class to define a decorator factory.  This class will need define the `__call__()` method, and may define any supporting methods that are useful.  Since it is a decorator factory, presumably there should be an `__init__()` method to process the parameterization.  The point to understand is that calling the class produces an instance, and that instance that acts as a decorator (and hence needs to be made callable as something that transforms a function in some manner).

In [38]:
from collections import defaultdict

class logfile(object):
    # Class-level attribute
    _logged_funcs = defaultdict(list)
    
    def __init__(self, filename):
        # Keep track of the name of the log file used
        self._logfile = filename
    
    @classmethod
    def _note_func(cls, logfile, func):
        cls._logged_funcs[logfile].append(func)
    
    def __call__(self, func):
        self._note_func(self._logfile, func)
        
        @wraps(func)
        def inner(*args, **kws):
            result = func(*args, **kws)
            now = datetime.utcnow().isoformat()
            name = func.__name__
            with open(self._logfile, 'a') as fh:
                fh.write(f"{now}\t{name}\t{args}\t{kws}\t{repr(result)}\n")
            return result
        return inner
    
    @classmethod
    def get_registry(cls):
        return dict(cls._logged_funcs)

In [49]:
@logfile('ine-lesson-en.log')
def g():
    "A function that says hello"
    return 'hello from function g'
g()

'hello from function g'

In [50]:
help(g)

Help on function g in module __main__:

g()
    A function that says hello



In [51]:
logfile.get_registry()

{'ine-lesson-en.log': [<function __main__.g()>,
  <function __main__.h()>,
  <function __main__.g()>],
 'ine-lesson-es.log': [<function __main__.i()>, <function __main__.j()>]}

In [52]:
@logfile('ine-lesson-en.log')
def h():
    return "bye from function h"
h()

'bye from function h'

In [53]:
@logfile('ine-lesson-es.log')
def i():
    return "Hola from the function i"
i()

'Hola from the function i'

In [54]:
@logfile('ine-lesson-es.log')
def j():
    return "Bye from the function j"
j()

'Bye from the function j'

In [45]:
g(), h(), i(), j()

('hello from function g',
 'bye from function h',
 'Hola de la función i',
 'Adiós de la función j')

In [46]:
logfile.get_registry()

{'ine-lesson-en.log': [<function __main__.g()>, <function __main__.h()>],
 'ine-lesson-es.log': [<function __main__.i()>, <function __main__.j()>]}

In [48]:
%%bash
cat ine-lesson*.log

2024-02-13T09:53:33.405067	g	()	{}	'hello from function g'
2024-02-13T09:53:35.180514	h	()	{}	'bye from function h'
2024-02-13T09:53:37.052404	g	()	{}	'hello from function g'
2024-02-13T09:53:37.052404	h	()	{}	'bye from function h'


Task exception was never retrieved
future: <Task finished name='Task-12' coro=<ScriptMagics.shebang.<locals>._handle_stream() done, defined at C:\Users\Laisha\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\script.py:211> exception=UnicodeDecodeError('utf-8', b"2024-02-13T09:53:35.666683\ti\t()\t{}\t'Hola de la funci\xf3n i'\r\n", 52, 53, 'invalid continuation byte')>
Traceback (most recent call last):
  File "C:\Users\Laisha\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\script.py", line 213, in _handle_stream
    line = (await stream.readline()).decode("utf8")
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 52: invalid continuation byte


# Memoization

Suppose we have a computationally intensive method, ```f()```, that calculates some result (in this case, a number).
We have to call this function many times, but do not wait forever to recalculate our result.

In [55]:
import time

def f(a, b):
    # <expensive number crunching here>
    out = a + b
    time.sleep(1.5)
    return out

In [56]:
%timeit f(3, 5)

1.51 s ± 985 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Assuming our function is *pure*, it would be nice to cache the results of our previous calls. This was, when `f()` is called again with the same arguments it will simply return the cached result instead of recalculating the answer.

This sort of caching is called *memoization*.  Lets define a class decorator that will memoize any function that we decorate. Note that there is a faster, better tested, and more flexible version of this particular decorator available as `functools.lru_cache` in the standard library.  As of Python 3.9, there is also a `functoools.cache()` as well (with slightly different behavior).

In [57]:
class Memoizer(object):
    def __init__(self, func):
        self.cache = {}
        self.func = func
        
    def __call__(self, *args, **kwargs):
        # Use sorted tuples because much smaller in memory than frozensets
        # Even though frozensets are slightly faster to construct.
        # We sort so that we can compare the keyword args.
        # We use strings because our args or kwargs may not be hashable.
        # Repr should return a unique string for its object
        key = (repr(args), repr(tuple(sorted(kwargs.items()))))
        if key in self.cache:
            return self.cache[key]
        self.cache[key] = self.func(*args, **kwargs)
        return self.cache[key]

In [58]:
import sys

@Memoizer
def f(a, b, mod=sys.maxsize):
    # <expensive number crunching here>
    out = (a + b) % mod
    time.sleep(1.5)
    return out

In [59]:
%time f(3, 5)

CPU times: total: 15.6 ms
Wall time: 1.51 s


8

In [60]:
%time f(6, 7, mod=37)

CPU times: total: 0 ns
Wall time: 1.51 s


13

In [61]:
%time f(3, 5)

CPU times: total: 0 ns
Wall time: 0 ns


8

In [62]:
%time f(6, 7, mod=37)

CPU times: total: 0 ns
Wall time: 0 ns


13

In [63]:
%time f(6, 7, mod=111)

CPU times: total: 0 ns
Wall time: 1.51 s


13

In [64]:
f.cache

{('(3, 5)', '()'): 8,
 ('(6, 7)', "(('mod', 37),)"): 13,
 ('(6, 7)', "(('mod', 111),)"): 13}

# Exercise

## Description

Writing parameterized decorators is a powerful way of describing some general cross-cutting behavior that is also partially specialized.  However, in decoratoring a function you have to determine if the "decorator" is actually a decorator or a decorator function.  At least you do in the examples in the lessons.

A number of widely used decorators in the standard library and in widely-used third-party tools can act as *either* a decorator factory *or* as a decorator directly.  For example:

```python
from functools import lru_cache

@lru_cache
def add1(a, b):
    return a+b

@lru_cache(maxsize=128)
def add2(a, b):
    return a+b
```

Both `add1()` and `add2()` are memoized, and in fact have the same maximum cache size.  In the factory variation, you could specify a non-default `maxsize`, of course.

For this exercise, you should write your own dual-use decorator.  Your will be called `@orbits` after the style of a Mandelbrot set function which resembles this:

```python
def mandelbrot(z0:complex, orbits:int=255) -> complex:
    z = z0
    for n in range(orbits):
        z = z**2 + z0
    return z
```

The idea here is that you would like to write a function that only expresses a single transformation of a (complex) number, but when decorated it will repeatedly apply that operation for a number of "orbits."  If not parameterized, it will default to 10 orbits.  As an example, the `m_orbit()` function is defined in the setup.

## Setup

In [84]:
# You may decide to use a class-based version.  
# Non-working identity decorator only created as example
def orbits(fn):
    def inner(z):
        return fn(z)
    return inner

# Definition of a single orbit
@orbits
def m_orbit(z: complex) -> complex:
    return z**2 + z

# correct is approx -0.0733106-0.0017179j
m_orbit(-0.6+0.3j)    #-> wrong is approx -0.33-0.06j

(-0.32999999999999996-0.06j)

## Solution

In [85]:
from types import FunctionType

def orbits(fun_or_count):
    if isinstance(fun_or_count, FunctionType):
        fn = fun_or_count
        def inner(z):
            for _ in range(10):
                z = fn(z)
            return z
        return inner
    
    elif isinstance(fun_or_count, int) and fun_or_count >= 0:
        n = fun_or_count
        def decorator(fn):
            def inner(z):
                for _ in range(n):
                    z = fn(z)
                return z
            return inner
        return decorator
    
    else:
        raise ValueError("Number of orbits must be a non-negative integer")

## Test Cases

In [86]:
def test_noparam():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits
    def j_orbit(z):
        return z**2 + c
    z = j_orbit(0.1-0.5j)
    assert isclose(z, -0.3170447+0.5734000j, abs_tol=1e-7), z
    
test_noparam()

In [87]:
def test_param0():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits(0)
    def j_orbit(z):
        return z**2 + c
    z = j_orbit(0.1-0.5j)    
    assert isclose(z, 0.1-0.5j), z
    
test_param0()

In [88]:
def test_param10():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits(10)
    def j_orbit(z):
        return z**2 + c
    z = j_orbit(0.1-0.5j)
    assert isclose(z, -0.3170447+0.5734000j, abs_tol=1e-7), z
    
test_param10()

In [89]:
def test_param255():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits(255)
    def j_orbit(z):
        return z**2 + c
    z = j_orbit(0.1-0.5j)
    assert isclose(z, -0.0304571+0.1706268j, abs_tol=1e-7), z
    
test_param255()

In [90]:
def test_default():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits(10)
    def j1_orbit(z): return z**2 + c
    @orbits
    def j2_orbit(z): return z**2 + c
    assert j1_orbit(0.1-0.5j) == j2_orbit(0.1-0.5j)
    
test_default()

-------------
Materials licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by the authors